Connect with databse postgres online

In [1]:
%pip install flask pandas sqlalchemy psycopg2 keplergl matplotlib

  Using cached flask-3.1.0-py3-none-any.whl (102 kB)
  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
  Using cached sqlalchemy-2.0.40-cp311-cp311-win_amd64.whl (2.1 MB)
  Using cached psycopg2-2.9.10-cp311-cp311-win_amd64.whl (1.2 MB)
  Using cached keplergl-0.3.7-py2.py3-none-any.whl
  Using cached matplotlib-3.10.1-cp311-cp311-win_amd64.whl (8.1 MB)
  Using cached werkzeug-3.1.3-py3-none-any.whl (224 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
  Using cached click-8.1.8-py3-none-any.whl (98 kB)
  Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
  Using cached numpy-2.2.4-cp311-cp311-win_amd64.whl (12.9 MB)
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
  Using cached greenlet-3.1.1-cp311-cp311-win_amd64.whl (298 kB)
     -------------------------------------- 139.8/139.8 kB 2.8 MB/s eta 0:00:00
  Using cached traittypes-0.2.


[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from flask import Flask, Response, render_template_string
import threading
import pandas as pd
from sqlalchemy import create_engine
from keplergl import KeplerGl
import matplotlib.pyplot as plt
import io
import base64

# === Database config ===
DB_USER = "testing1_415q_user"
DB_PASSWORD = "syezjh72qciEKZUCBIcR4LF6YkiH7aXK"
DB_HOST = "dpg-cvr1u8ngi27c738j3acg-a.singapore-postgres.render.com"
DB_PORT = "5432"
DB_NAME = "testing1_415q"

# === Connect to DB ===
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
query = "SELECT name, amenity, latitude, longitude FROM health_facilities;"

# === Flask app ===
app = Flask(__name__)

# === Map config ===
config = {
    'version': 'v1',
    'config': {
        'mapState': {
            'latitude': 3.944035,
            'longitude': 102.638624,
            'zoom': 4.32
        }
    }
}

# === Function to create donut chart ===
def create_donut_chart():
    df = pd.read_sql(query, engine)
    amenities_count = df['amenity'].value_counts()

    # Create donut chart
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.pie(amenities_count, labels=amenities_count.index, autopct='%1.1f%%', startangle=90, wedgeprops={'width': 0.4})
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

    # Save it to a BytesIO object and convert to base64
    img_bytes = io.BytesIO()
    plt.savefig(img_bytes, format='png')
    img_bytes.seek(0)
    img_base64 = base64.b64encode(img_bytes.read()).decode('utf-8')

    return img_base64

# === Main route with buttons ===
@app.route('/')
def index():
    # Generate the donut chart as a base64 string
    chart_image = create_donut_chart()

    # Embed the base64 chart image into the HTML template
    html = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Health Facilities Map</title>
    <style>
        * {{
            box-sizing: border-box;
        }}
        body {{
            margin: 0;
            padding: 0;
            font-family: sans-serif;
            text-align: center;
            background-color: #f4f4f4;
            height: 100vh;
            overflow-x: hidden; /* Prevent horizontal scrolling */
        }}
        .container {{
            display: flex;
            justify-content: space-between;
            align-items: flex-start;
            width: 100%;
            padding: 10px;
            flex-direction: column; /* Arrange the elements vertically */
        }}
        iframe {{
            width: 70vw; /* Adjust the width to 70% of the viewport */
            height: 70vh; /* Height is 60% of the viewport height */
            border: none;
        }}
        .button-container {{
            display: flex;
            justify-content: center;
            margin-top: 20px;
        }}
        .chart-container {{
            width: 70vw; 
            height: 80vh; 
            padding: 20px;
            background-color: #fff;
            border-radius: 8px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            overflow-y: auto;
            margin-top: 20px;
        }}
        .btn {{
            padding: 10px 20px;
            font-size: 16px;
            cursor: pointer;
            background-color: #007BFF;
            color: white;
            border: 2px solid #0056b3;
            border-radius: 5px;
            margin: 0 10px;
        }}
        .btn:hover {{
            background-color: #0056b3;
        }}
        #kepler-gl__map {{
            width: 100% !important;
            height: 100vh !important;
        }}
    </style>
</head>
<body>
    <div class="container">
        <iframe id="mapFrame" src="/map"></iframe>
        
        <div class="button-container">
            <button class="btn" onclick="refreshMap()">Refresh Map</button>
            <button class="btn" onclick="showClinicsOnly()">Show Clinics Only</button>
        </div>
        
        <div class="chart-container">
            <h3>Amenity Distribution</h3>
            <img src="data:image/png;base64,{chart_image}" alt="Donut Chart">
        </div>
    </div>

    <!-- The script block with functions -->
    <script>
        function refreshMap() {{
            document.getElementById("mapFrame").src = "/map?" + new Date().getTime();
        }}

        function showClinicsOnly() {{
            document.getElementById("mapFrame").src = "/map/clinic?" + new Date().getTime();
        }}
    </script>
</body>
</html>
"""
    return render_template_string(html)


# === Default full map ===
@app.route('/map')
def map_all():
    df = pd.read_sql(query, engine)
    map_ = KeplerGl(height=600)
    map_.add_data(data=df, name='All Health Facilities')
    map_.config = config
    return Response(map_._repr_html_(), mimetype='text/html')

# === Filtered clinic-only map ===
@app.route('/map/clinic')
def map_clinic():
    df = pd.read_sql(query, engine)
    clinic_df = df[df['amenity'].str.lower() == 'clinic']  # Filter for clinics only
    map_ = KeplerGl(height=600)
    map_.add_data(data=clinic_df, name='Clinics Only')
    map_.config = config
    return Response(map_._repr_html_(), mimetype='text/html')

# === Run Flask ===
def run_flask():
    app.run(debug=True, port=5006, use_reloader=False)

threading.Thread(target=run_flask).start()


 * Serving Flask app '__main__'


 * Debug mode: on


 * Running on http://127.0.0.1:5006
Press CTRL+C to quit
127.0.0.1 - - [11/Apr/2025 10:59:03] "GET / HTTP/1.1" 200 -


User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


127.0.0.1 - - [11/Apr/2025 10:59:04] "GET /map HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2025 10:59:07] "GET /favicon.ico HTTP/1.1" 404 -
